# 04. LangGraph Multi-Agent Systems

## Mục tiêu học tập

Trong notebook này, bạn sẽ học cách:
- Hiểu khái niệm và kiến trúc của hệ thống đa tác nhân (Multi-Agent Systems)
- Tạo và quản lý nhiều tác nhân với vai trò và chức năng riêng biệt
- Xây dựng các luồng tương tác và phối hợp giữa các tác nhân
- Quản lý trạng thái chung và truyền thông tin giữa các tác nhân
- Triển khai các mô hình hợp tác khác nhau trong hệ thống đa tác nhân

## Giới thiệu về Hệ thống Đa Tác nhân

### Khái niệm
Hệ thống đa tác nhân (Multi-Agent Systems - MAS) là một mô hình tính toán trong đó nhiều tác nhân tự động tương tác và hợp tác để giải quyết các vấn đề phức tạp. Mỗi tác nhân có thể có:
- Chuyên môn riêng biệt
- Tập hợp công cụ khác nhau
- Phong cách giao tiếp riêng
- Quyền hạn và trách nhiệm cụ thể

### Lợi ích
- **Modular**: Chia nhỏ vấn đề phức tạp thành các phần có thể quản lý
- **Scalable**: Dễ dàng thêm hoặc bớt tác nhân theo nhu cầu
- **Specialized**: Mỗi tác nhân có thể được tối ưu hóa cho một nhiệm vụ cụ thể
- **Fault-tolerant**: Hệ thống có thể tiếp tục hoạt động khi một tác nhân gặp sự cố

### Thách thức
- **Coordination**: Đảm bảo các tác nhân làm việc hiệu quả với nhau
- **Communication**: Quản lý việc truyền thông tin giữa các tác nhân
- **State Management**: Duy trì tính nhất quán của trạng thái chung
- **Conflict Resolution**: Xử lý các xung đột giữa các tác nhân

## Cài đặt & Cấu hình

In [ ]:
# Cài đặt các thư viện cần thiết
!pip install -q langgraph langchain-anthropic python-dotenv

import os
from dotenv import load_dotenv
from typing import TypedDict, List, Dict, Any
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
import json

# Load environment variables
load_dotenv()

# Cấu hình LLM
llm = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    temperature=0.7,
    api_key=os.getenv("ANTHROPIC_API_KEY")
)

print("✅ Cài đặt hoàn tất!")

## Ví dụ 1: Hệ thống Research & Writing

Chúng ta sẽ tạo một hệ thống với hai tác nhân:
- **Researcher**: Chuyên về nghiên cứu và thu thập thông tin
- **Writer**: Chuyên về viết và trình bày nội dung

In [ ]:
# Định nghĩa State cho hệ thống đa tác nhân
class MultiAgentState(TypedDict):
    messages: List[Dict[str, Any]]
    current_agent: str
    topic: str
    research_data: str
    final_content: str
    step: int
    max_steps: int

# Tác nhân Researcher
def researcher_agent(state: MultiAgentState) -> MultiAgentState:
    """Tác nhân nghiên cứu - thu thập và phân tích thông tin"""
    
    topic = state["topic"]
    
    # System prompt cho Researcher
    research_prompt = f"""
    Bạn là một nhà nghiên cứu chuyên nghiệp. Nhiệm vụ của bạn là:
    1. Phân tích chủ đề: {topic}
    2. Xác định các khía cạnh quan trọng cần nghiên cứu
    3. Cung cấp thông tin chi tiết, có cấu trúc về chủ đề
    4. Đưa ra các điểm chính và thống kê (nếu có)
    
    Hãy cung cấp một báo cáo nghiên cứu ngắn gọn nhưng đầy đủ về chủ đề này.
    """
    
    messages = [
        SystemMessage(content=research_prompt),
        HumanMessage(content=f"Hãy nghiên cứu về: {topic}")
    ]
    
    response = llm.invoke(messages)
    research_data = response.content
    
    # Cập nhật state
    new_message = {
        "agent": "researcher",
        "content": research_data,
        "step": state["step"]
    }
    
    return {
        **state,
        "messages": state["messages"] + [new_message],
        "research_data": research_data,
        "current_agent": "writer",
        "step": state["step"] + 1
    }

# Tác nhân Writer
def writer_agent(state: MultiAgentState) -> MultiAgentState:
    """Tác nhân viết - tạo nội dung dựa trên nghiên cứu"""
    
    topic = state["topic"]
    research_data = state["research_data"]
    
    # System prompt cho Writer
    writing_prompt = f"""
    Bạn là một nhà văn chuyên nghiệp. Nhiệm vụ của bạn là:
    1. Sử dụng dữ liệu nghiên cứu đã được cung cấp
    2. Tạo ra một bài viết hấp dẫn và dễ hiểu
    3. Có cấu trúc rõ ràng với phần mở đầu, nội dung chính và kết luận
    4. Sử dụng ngôn ngữ phù hợp với độc giả
    
    Dữ liệu nghiên cứu:
    {research_data}
    
    Hãy viết một bài viết hoàn chỉnh về chủ đề: {topic}
    """
    
    messages = [
        SystemMessage(content=writing_prompt),
        HumanMessage(content=f"Viết bài về: {topic}")
    ]
    
    response = llm.invoke(messages)
    final_content = response.content
    
    # Cập nhật state
    new_message = {
        "agent": "writer",
        "content": final_content,
        "step": state["step"]
    }
    
    return {
        **state,
        "messages": state["messages"] + [new_message],
        "final_content": final_content,
        "current_agent": "completed",
        "step": state["step"] + 1
    }

print("✅ Định nghĩa các tác nhân hoàn tất!")

In [ ]:
# Hàm điều hướng để quyết định tác nhân tiếp theo
def router(state: MultiAgentState) -> str:
    """Quyết định tác nhân nào sẽ được thực thi tiếp theo"""
    
    current_agent = state["current_agent"]
    step = state["step"]
    max_steps = state["max_steps"]
    
    # Kiểm tra điều kiện dừng
    if step >= max_steps or current_agent == "completed":
        return END
    
    # Điều hướng dựa trên current_agent
    if current_agent == "researcher":
        return "researcher"
    elif current_agent == "writer":
        return "writer"
    else:
        return END

# Tạo StateGraph
def create_research_writing_graph():
    """Tạo graph cho hệ thống research & writing"""
    
    workflow = StateGraph(MultiAgentState)
    
    # Thêm các nodes (tác nhân)
    workflow.add_node("researcher", researcher_agent)
    workflow.add_node("writer", writer_agent)
    
    # Thiết lập điểm bắt đầu
    workflow.set_entry_point("researcher")
    
    # Thêm conditional edges
    workflow.add_conditional_edges(
        "researcher",
        router,
        {
            "writer": "writer",
            END: END
        }
    )
    
    workflow.add_conditional_edges(
        "writer",
        router,
        {
            END: END
        }
    )
    
    # Compile graph
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    
    return app

print("✅ Graph được tạo thành công!")

In [ ]:
# Test hệ thống Research & Writing
def test_research_writing_system():
    """Test hệ thống đa tác nhân research & writing"""
    
    app = create_research_writing_graph()
    
    # Khởi tạo state
    initial_state = {
        "messages": [],
        "current_agent": "researcher",
        "topic": "Trí tuệ nhân tạo trong giáo dục",
        "research_data": "",
        "final_content": "",
        "step": 0,
        "max_steps": 5
    }
    
    # Thực thi workflow
    config = {"configurable": {"thread_id": "research_writing_demo"}}
    
    print("🚀 Bắt đầu quá trình research & writing...\n")
    
    # Chạy workflow
    result = app.invoke(initial_state, config)
    
    # Hiển thị kết quả
    print("📊 === KẾT QUẢ NGHIÊN CỨU ===")
    print(f"Agent: Researcher")
    print(f"Nội dung: {result['research_data'][:500]}...\n")
    
    print("✍️ === BÀI VIẾT HOÀN CHỈNH ===")
    print(f"Agent: Writer")
    print(f"Nội dung: {result['final_content'][:500]}...\n")
    
    print(f"📈 Tổng số bước thực hiện: {result['step']}")
    print(f"📝 Tổng số messages: {len(result['messages'])}")
    
    return result

# Chạy test
result1 = test_research_writing_system()

## Ví dụ 2: Hệ thống Code Review

Tạo một hệ thống với ba tác nhân để review code:
- **Analyzer**: Phân tích cấu trúc và logic code
- **Security Reviewer**: Kiểm tra các vấn đề bảo mật
- **Summarizer**: Tổng hợp và đưa ra kết luận cuối cùng

In [ ]:
# State cho hệ thống Code Review
class CodeReviewState(TypedDict):
    messages: List[Dict[str, Any]]
    current_agent: str
    code_snippet: str
    analysis_report: str
    security_report: str
    final_review: str
    step: int
    max_steps: int

# Analyzer Agent
def analyzer_agent(state: CodeReviewState) -> CodeReviewState:
    """Tác nhân phân tích code - kiểm tra cấu trúc và logic"""
    
    code = state["code_snippet"]
    
    analysis_prompt = f"""
    Bạn là một chuyên gia phân tích code. Hãy phân tích đoạn code sau theo các tiêu chí:
    
    1. **Cấu trúc code**: Tổ chức, readability, naming conventions
    2. **Logic**: Tính chính xác, hiệu quả của thuật toán
    3. **Best Practices**: Tuân thủ các nguyên tắc lập trình
    4. **Performance**: Các vấn đề hiệu suất tiềm ẩn
    5. **Maintainability**: Khả năng bảo trì và mở rộng
    
    Code cần phân tích:
    ```
    {code}
    ```
    
    Hãy đưa ra một báo cáo chi tiết với điểm mạnh, điểm yếu và đề xuất cải thiện.
    """
    
    messages = [
        SystemMessage(content=analysis_prompt),
        HumanMessage(content="Phân tích code này")
    ]
    
    response = llm.invoke(messages)
    analysis_report = response.content
    
    new_message = {
        "agent": "analyzer",
        "content": analysis_report,
        "step": state["step"]
    }
    
    return {
        **state,
        "messages": state["messages"] + [new_message],
        "analysis_report": analysis_report,
        "current_agent": "security_reviewer",
        "step": state["step"] + 1
    }

# Security Reviewer Agent
def security_reviewer_agent(state: CodeReviewState) -> CodeReviewState:
    """Tác nhân review bảo mật - kiểm tra các lỗ hổng bảo mật"""
    
    code = state["code_snippet"]
    
    security_prompt = f"""
    Bạn là một chuyên gia bảo mật ứng dụng. Hãy kiểm tra đoạn code sau về các vấn đề bảo mật:
    
    1. **Input Validation**: Kiểm tra input từ người dùng
    2. **SQL Injection**: Các lỗ hổng SQL injection
    3. **XSS**: Cross-site scripting vulnerabilities
    4. **Authentication & Authorization**: Vấn đề xác thực và phân quyền
    5. **Data Exposure**: Rò rỉ thông tin nhạy cảm
    6. **Error Handling**: Xử lý lỗi an toàn
    
    Code cần kiểm tra:
    ```
    {code}
    ```
    
    Hãy đưa ra báo cáo bảo mật với mức độ nghiêm trọng và cách khắc phục.
    """
    
    messages = [
        SystemMessage(content=security_prompt),
        HumanMessage(content="Kiểm tra bảo mật code này")
    ]
    
    response = llm.invoke(messages)
    security_report = response.content
    
    new_message = {
        "agent": "security_reviewer",
        "content": security_report,
        "step": state["step"]
    }
    
    return {
        **state,
        "messages": state["messages"] + [new_message],
        "security_report": security_report,
        "current_agent": "summarizer",
        "step": state["step"] + 1
    }

# Summarizer Agent
def summarizer_agent(state: CodeReviewState) -> CodeReviewState:
    """Tác nhân tổng hợp - tạo báo cáo review cuối cùng"""
    
    analysis = state["analysis_report"]
    security = state["security_report"]
    
    summary_prompt = f"""
    Bạn là một lead developer chịu trách nhiệm tổng hợp các báo cáo code review.
    
    Báo cáo phân tích:
    {analysis}
    
    Báo cáo bảo mật:
    {security}
    
    Hãy tạo một báo cáo tổng hợp cuối cùng bao gồm:
    1. **Tổng quan**: Đánh giá chung về chất lượng code
    2. **Điểm mạnh**: Các điểm tích cực
    3. **Vấn đề quan trọng**: Các vấn đề cần ưu tiên khắc phục
    4. **Khuyến nghị**: Các hành động cụ thể cần thực hiện
    5. **Quyết định**: Approve/Request Changes/Reject
    
    Hãy đưa ra một báo cáo ngắn gọn và có tính hành động cao.
    """
    
    messages = [
        SystemMessage(content=summary_prompt),
        HumanMessage(content="Tổng hợp báo cáo code review")
    ]
    
    response = llm.invoke(messages)
    final_review = response.content
    
    new_message = {
        "agent": "summarizer",
        "content": final_review,
        "step": state["step"]
    }
    
    return {
        **state,
        "messages": state["messages"] + [new_message],
        "final_review": final_review,
        "current_agent": "completed",
        "step": state["step"] + 1
    }

print("✅ Định nghĩa các tác nhân code review hoàn tất!")

In [ ]:
# Router cho hệ thống Code Review
def code_review_router(state: CodeReviewState) -> str:
    """Điều hướng cho workflow code review"""
    
    current_agent = state["current_agent"]
    step = state["step"]
    max_steps = state["max_steps"]
    
    if step >= max_steps or current_agent == "completed":
        return END
    
    if current_agent == "analyzer":
        return "analyzer"
    elif current_agent == "security_reviewer":
        return "security_reviewer"
    elif current_agent == "summarizer":
        return "summarizer"
    else:
        return END

# Tạo Code Review Graph
def create_code_review_graph():
    """Tạo graph cho hệ thống code review"""
    
    workflow = StateGraph(CodeReviewState)
    
    # Thêm các nodes
    workflow.add_node("analyzer", analyzer_agent)
    workflow.add_node("security_reviewer", security_reviewer_agent)
    workflow.add_node("summarizer", summarizer_agent)
    
    # Thiết lập entry point
    workflow.set_entry_point("analyzer")
    
    # Thêm edges
    workflow.add_conditional_edges(
        "analyzer",
        code_review_router,
        {
            "security_reviewer": "security_reviewer",
            END: END
        }
    )
    
    workflow.add_conditional_edges(
        "security_reviewer",
        code_review_router,
        {
            "summarizer": "summarizer",
            END: END
        }
    )
    
    workflow.add_conditional_edges(
        "summarizer",
        code_review_router,
        {
            END: END
        }
    )
    
    # Compile
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    
    return app

print("✅ Code Review Graph được tạo thành công!")

In [ ]:
# Test hệ thống Code Review
def test_code_review_system():
    """Test hệ thống đa tác nhân code review"""
    
    app = create_code_review_graph()
    
    # Sample code để review
    sample_code = """
def login(username, password):
    import sqlite3
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()
    
    # Kiểm tra user trong database
    query = f"SELECT * FROM users WHERE username = '{username}' AND password = '{password}'"
    cursor.execute(query)
    user = cursor.fetchone()
    
    if user:
        return {"success": True, "user_id": user[0], "username": user[1]}
    else:
        return {"success": False, "error": "Invalid credentials"}
    """
    
    # Khởi tạo state
    initial_state = {
        "messages": [],
        "current_agent": "analyzer",
        "code_snippet": sample_code,
        "analysis_report": "",
        "security_report": "",
        "final_review": "",
        "step": 0,
        "max_steps": 10
    }
    
    # Thực thi workflow
    config = {"configurable": {"thread_id": "code_review_demo"}}
    
    print("🔍 Bắt đầu quá trình code review...\n")
    print("📝 Code cần review:")
    print(sample_code)
    print("\n" + "="*50 + "\n")
    
    # Chạy workflow
    result = app.invoke(initial_state, config)
    
    # Hiển thị kết quả
    print("🔬 === BÁO CÁO PHÂN TÍCH ===")
    print(result['analysis_report'][:600] + "...\n")
    
    print("🔒 === BÁO CÁO BẢO MẬT ===")
    print(result['security_report'][:600] + "...\n")
    
    print("📋 === BÁO CÁO TỔNG HỢP ===")
    print(result['final_review'][:600] + "...\n")
    
    print(f"📊 Tổng số bước thực hiện: {result['step']}")
    print(f"👥 Số tác nhân tham gia: {len(result['messages'])}")
    
    return result

# Chạy test
result2 = test_code_review_system()

## Ví dụ 3: Hệ thống Phối hợp Phức tạp với Điều kiện

Tạo một hệ thống phức tạp hơn với khả năng:
- Các tác nhân có thể gọi lại nhau
- Điều kiện phức tạp để quyết định luồng
- Xử lý song song và tuần tự

In [ ]:
# State cho hệ thống phối hợp phức tạp
class ComplexCoordinationState(TypedDict):
    messages: List[Dict[str, Any]]
    current_agent: str
    task_description: str
    planner_output: str
    executor_outputs: List[str]
    reviewer_feedback: str
    iterations: int
    max_iterations: int
    is_approved: bool
    step: int

# Planner Agent
def planner_agent(state: ComplexCoordinationState) -> ComplexCoordinationState:
    """Tác nhân lập kế hoạch - phân tích và lập kế hoạch thực hiện"""
    
    task = state["task_description"]
    iteration = state["iterations"]
    feedback = state.get("reviewer_feedback", "")
    
    if iteration > 0 and feedback:
        planning_prompt = f"""
        Bạn là một chuyên gia lập kế hoạch. Dựa trên feedback sau, hãy điều chỉnh kế hoạch:
        
        Nhiệm vụ gốc: {task}
        Feedback từ reviewer: {feedback}
        Lần lặp thứ: {iteration + 1}
        
        Hãy tạo một kế hoạch chi tiết được cải thiện với các bước cụ thể.
        """
    else:
        planning_prompt = f"""
        Bạn là một chuyên gia lập kế hoạch. Hãy phân tích nhiệm vụ sau và tạo kế hoạch thực hiện:
        
        Nhiệm vụ: {task}
        
        Hãy tạo một kế hoạch chi tiết với:
        1. Phân tích yêu cầu
        2. Các bước thực hiện cụ thể
        3. Tiêu chí đánh giá thành công
        4. Rủi ro và cách xử lý
        """
    
    messages = [
        SystemMessage(content=planning_prompt),
        HumanMessage(content="Tạo kế hoạch thực hiện")
    ]
    
    response = llm.invoke(messages)
    planner_output = response.content
    
    new_message = {
        "agent": "planner",
        "content": planner_output,
        "iteration": iteration,
        "step": state["step"]
    }
    
    return {
        **state,
        "messages": state["messages"] + [new_message],
        "planner_output": planner_output,
        "current_agent": "executor",
        "step": state["step"] + 1
    }

# Executor Agent
def executor_agent(state: ComplexCoordinationState) -> ComplexCoordinationState:
    """Tác nhân thực thi - thực hiện theo kế hoạch"""
    
    plan = state["planner_output"]
    
    execution_prompt = f"""
    Bạn là một chuyên gia thực thi. Hãy thực hiện theo kế hoạch đã được đưa ra:
    
    Kế hoạch:
    {plan}
    
    Hãy thực hiện từng bước trong kế hoạch và báo cáo kết quả chi tiết.
    Bao gồm:
    1. Các bước đã thực hiện
    2. Kết quả đạt được
    3. Khó khăn gặp phải (nếu có)
    4. Đề xuất cải thiện
    """
    
    messages = [
        SystemMessage(content=execution_prompt),
        HumanMessage(content="Thực hiện kế hoạch")
    ]
    
    response = llm.invoke(messages)
    execution_result = response.content
    
    # Thêm kết quả vào danh sách
    new_outputs = state["executor_outputs"] + [execution_result]
    
    new_message = {
        "agent": "executor",
        "content": execution_result,
        "iteration": state["iterations"],
        "step": state["step"]
    }
    
    return {
        **state,
        "messages": state["messages"] + [new_message],
        "executor_outputs": new_outputs,
        "current_agent": "reviewer",
        "step": state["step"] + 1
    }

# Reviewer Agent
def reviewer_agent(state: ComplexCoordinationState) -> ComplexCoordinationState:
    """Tác nhân đánh giá - review và quyết định có cần lặp lại không"""
    
    plan = state["planner_output"]
    execution = state["executor_outputs"][-1]  # Lấy kết quả thực thi mới nhất
    iteration = state["iterations"]
    
    review_prompt = f"""
    Bạn là một chuyên gia đánh giá chất lượng. Hãy review kết quả thực hiện:
    
    Kế hoạch ban đầu:
    {plan}
    
    Kết quả thực hiện:
    {execution}
    
    Lần lặp hiện tại: {iteration + 1}
    
    Hãy đánh giá:
    1. Mức độ hoàn thành nhiệm vụ (0-100%)
    2. Chất lượng kết quả
    3. Có cần cải thiện gì không?
    4. Quyết định: APPROVE (chấp nhận) hoặc REVISE (cần sửa đổi)
    
    Kết thúc bằng từ khóa: APPROVE hoặc REVISE
    """
    
    messages = [
        SystemMessage(content=review_prompt),
        HumanMessage(content="Đánh giá kết quả thực hiện")
    ]
    
    response = llm.invoke(messages)
    review_feedback = response.content
    
    # Kiểm tra quyết định
    is_approved = "APPROVE" in review_feedback.upper()
    next_agent = "completed" if is_approved else "planner"
    
    new_message = {
        "agent": "reviewer",
        "content": review_feedback,
        "iteration": iteration,
        "step": state["step"],
        "decision": "APPROVE" if is_approved else "REVISE"
    }
    
    return {
        **state,
        "messages": state["messages"] + [new_message],
        "reviewer_feedback": review_feedback,
        "is_approved": is_approved,
        "current_agent": next_agent,
        "iterations": iteration + 1,
        "step": state["step"] + 1
    }

print("✅ Định nghĩa các tác nhân phối hợp phức tạp hoàn tất!")

In [ ]:
# Router cho hệ thống phối hợp phức tạp
def complex_coordination_router(state: ComplexCoordinationState) -> str:
    """Điều hướng phức tạp với các điều kiện"""
    
    current_agent = state["current_agent"]
    iterations = state["iterations"]
    max_iterations = state["max_iterations"]
    is_approved = state["is_approved"]
    
    # Kiểm tra điều kiện dừng
    if current_agent == "completed" or is_approved:
        return END
    
    if iterations >= max_iterations:
        return END
    
    # Điều hướng theo agent hiện tại
    if current_agent == "planner":
        return "planner"
    elif current_agent == "executor":
        return "executor"
    elif current_agent == "reviewer":
        return "reviewer"
    else:
        return END

# Tạo Complex Coordination Graph
def create_complex_coordination_graph():
    """Tạo graph cho hệ thống phối hợp phức tạp"""
    
    workflow = StateGraph(ComplexCoordinationState)
    
    # Thêm các nodes
    workflow.add_node("planner", planner_agent)
    workflow.add_node("executor", executor_agent)
    workflow.add_node("reviewer", reviewer_agent)
    
    # Thiết lập entry point
    workflow.set_entry_point("planner")
    
    # Thêm conditional edges cho tất cả nodes
    for node in ["planner", "executor", "reviewer"]:
        workflow.add_conditional_edges(
            node,
            complex_coordination_router,
            {
                "planner": "planner",
                "executor": "executor",
                "reviewer": "reviewer",
                END: END
            }
        )
    
    # Compile
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    
    return app

print("✅ Complex Coordination Graph được tạo thành công!")

In [ ]:
# Test hệ thống phối hợp phức tạp
def test_complex_coordination_system():
    """Test hệ thống đa tác nhân phối hợp phức tạp"""
    
    app = create_complex_coordination_graph()
    
    # Khởi tạo state
    initial_state = {
        "messages": [],
        "current_agent": "planner",
        "task_description": "Xây dựng một hệ thống chatbot AI cho dịch vụ khách hàng của một ngân hàng",
        "planner_output": "",
        "executor_outputs": [],
        "reviewer_feedback": "",
        "iterations": 0,
        "max_iterations": 3,
        "is_approved": False,
        "step": 0
    }
    
    # Thực thi workflow
    config = {"configurable": {"thread_id": "complex_coordination_demo"}}
    
    print("🎯 Bắt đầu quá trình phối hợp phức tạp...\n")
    print(f"📋 Nhiệm vụ: {initial_state['task_description']}\n")
    print("="*60 + "\n")
    
    # Chạy workflow
    result = app.invoke(initial_state, config)
    
    # Hiển thị kết quả chi tiết
    print("📈 === TỔNG QUAN QUÁ TRÌNH ===")
    print(f"Số lần lặp: {result['iterations']}")
    print(f"Tổng số bước: {result['step']}")
    print(f"Kết quả cuối: {'✅ Được chấp nhận' if result['is_approved'] else '❌ Cần cải thiện'}\n")
    
    # Hiển thị các bước thực hiện
    current_iteration = -1
    for msg in result['messages']:
        if msg.get('iteration', 0) != current_iteration:
            current_iteration = msg.get('iteration', 0)
            print(f"\n🔄 === LẦN LẶP {current_iteration + 1} ===")
        
        agent_emoji = {
            'planner': '📋',
            'executor': '⚡',
            'reviewer': '🔍'
        }
        
        emoji = agent_emoji.get(msg['agent'], '🤖')
        print(f"\n{emoji} {msg['agent'].upper()}:")
        content = msg['content'][:300] + "..." if len(msg['content']) > 300 else msg['content']
        print(content)
        
        if msg['agent'] == 'reviewer' and 'decision' in msg:
            decision_emoji = '✅' if msg['decision'] == 'APPROVE' else '🔄'
            print(f"\n{decision_emoji} Quyết định: {msg['decision']}")
    
    return result

# Chạy test
result3 = test_complex_coordination_system()

## Giải thích & Phân tích

### Cách các tác nhân tương tác

1. **Sequential Processing**: Các tác nhân làm việc tuần tự, mỗi tác nhân nhận kết quả từ tác nhân trước đó
2. **State Sharing**: Tất cả tác nhân chia sẻ chung một state, cho phép truyền thông tin hiệu quả
3. **Conditional Routing**: Sử dụng router functions để quyết định tác nhân nào sẽ được thực thi tiếp theo
4. **Feedback Loops**: Hệ thống phối hợp phức tạp cho phép tác nhân quay lại các bước trước đó

### Quản lý luồng điều khiển

1. **Entry Points**: Xác định điểm bắt đầu của workflow
2. **Conditional Edges**: Sử dụng điều kiện để quyết định luồng tiếp theo
3. **State-based Routing**: Routing dựa trên trạng thái hiện tại của hệ thống
4. **Termination Conditions**: Điều kiện dừng rõ ràng để tránh vòng lặp vô hạn

### Truyền thông tin giữa các tác nhân

1. **Shared State**: Sử dụng TypedDict để định nghĩa cấu trúc state chung
2. **Message History**: Lưu trữ lịch sử các message để theo dõi quá trình
3. **Specialized Fields**: Các trường chuyên biệt cho từng loại thông tin
4. **Context Preservation**: Bảo toàn context qua các bước thực thi

### Lợi ích của kiến trúc đa tác nhân

1. **Separation of Concerns**: Mỗi tác nhân có trách nhiệm riêng biệt
2. **Reusability**: Có thể tái sử dụng tác nhân trong các workflow khác
3. **Testability**: Dễ dàng test từng tác nhân độc lập
4. **Scalability**: Có thể thêm hoặc bớt tác nhân theo nhu cầu
5. **Maintainability**: Dễ bảo trì và cập nhật từng phần

## Các Pattern và Best Practices

### 1. Pattern thiết kế tác nhân
- **Single Responsibility**: Mỗi tác nhân có một chức năng chính
- **Clear Interface**: Định nghĩa rõ ràng input và output
- **Error Handling**: Xử lý lỗi gracefully

### 2. State Management
- **Immutable Updates**: Cập nhật state một cách bất biến
- **Validation**: Kiểm tra tính hợp lệ của state
- **Documentation**: Ghi chú rõ ràng về cấu trúc state

### 3. Routing Strategies
- **Simple Sequential**: Tác nhân A → B → C
- **Conditional**: Dựa trên điều kiện để chọn tác nhân
- **Parallel**: Nhiều tác nhân chạy song song
- **Iterative**: Lặp lại cho đến khi đạt điều kiện

### 4. Communication Patterns
- **Request-Response**: Tác nhân A gửi yêu cầu, tác nhân B phản hồi
- **Pipeline**: Dữ liệu chảy qua chuỗi tác nhân
- **Publish-Subscribe**: Tác nhân subscribe vào các event
- **Shared Memory**: Chia sẻ thông tin qua state chung

In [ ]:
# Bonus: Visualization của workflow
def visualize_workflow_execution(result, title="Workflow Execution"):
    """Tạo visualization đơn giản cho workflow execution"""
    
    print(f"\n🎨 === {title.upper()} VISUALIZATION ===")
    print("\n" + "─" * 60)
    
    messages = result.get('messages', [])
    
    agent_colors = {
        'planner': '🟦',
        'executor': '🟩', 
        'reviewer': '🟨',
        'researcher': '🟪',
        'writer': '🟫',
        'analyzer': '⬜',
        'security_reviewer': '🟥',
        'summarizer': '🟧'
    }
    
    timeline = []
    for i, msg in enumerate(messages):
        agent = msg['agent']
        color = agent_colors.get(agent, '⚫')
        timeline.append(f"{color} {agent}")
    
    print("Timeline: " + " → ".join(timeline))
    
    # Statistics
    agent_counts = {}
    for msg in messages:
        agent = msg['agent']
        agent_counts[agent] = agent_counts.get(agent, 0) + 1
    
    print("\n📊 Agent Activity:")
    for agent, count in agent_counts.items():
        color = agent_colors.get(agent, '⚫')
        bar = "█" * count
        print(f"  {color} {agent:<15}: {bar} ({count})")
    
    print("\n" + "─" * 60)

# Visualize kết quả các test
print("🎭 VISUALIZATION OF ALL WORKFLOWS")
visualize_workflow_execution(result1, "Research & Writing System")
visualize_workflow_execution(result2, "Code Review System")
visualize_workflow_execution(result3, "Complex Coordination System")

## Tài liệu tham khảo

### LangGraph Documentation
- [Multi-Agent Systems](https://langchain-ai.github.io/langgraph/tutorials/multi_agent/)
- [Agent Supervisor](https://langchain-ai.github.io/langgraph/tutorials/multi_agent/agent_supervisor/)
- [Hierarchical Agent Teams](https://langchain-ai.github.io/langgraph/tutorials/multi_agent/hierarchical_agent_teams/)
- [StateGraph API Reference](https://langchain-ai.github.io/langgraph/reference/graphs/)
- [Conditional Edges](https://langchain-ai.github.io/langgraph/how-tos/branching/)

### Advanced Topics
- [Human-in-the-Loop](https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/)
- [Persistence and Memory](https://langchain-ai.github.io/langgraph/how-tos/persistence/)
- [Streaming](https://langchain-ai.github.io/langgraph/how-tos/streaming/)
- [Error Handling](https://langchain-ai.github.io/langgraph/how-tos/error-handling/)

### Examples and Tutorials
- [Multi-Agent Examples Repository](https://github.com/langchain-ai/langgraph/tree/main/examples)
- [LangGraph Cookbook](https://langchain-ai.github.io/langgraph/tutorials/)
- [Best Practices Guide](https://langchain-ai.github.io/langgraph/concepts/)

## Kết luận & Bước tiếp theo

### Tóm tắt
Trong notebook này, chúng ta đã học cách:
- Thiết kế và triển khai hệ thống đa tác nhân với LangGraph
- Quản lý luồng thực thi và truyền thông tin giữa các tác nhân
- Xây dựng các pattern phối hợp khác nhau: sequential, conditional, và iterative
- Sử dụng state management hiệu quả cho hệ thống phức tạp

### Kỹ năng đã phát triển
1. **Agent Design**: Thiết kế tác nhân với chức năng chuyên biệt
2. **Workflow Orchestration**: Điều phối workflow phức tạp
3. **State Management**: Quản lý trạng thái chia sẻ
4. **Error Handling**: Xử lý lỗi và edge cases
5. **System Integration**: Tích hợp nhiều thành phần

### Bước tiếp theo
1. **Advanced Patterns**: Học các pattern phức tạp hơn như hierarchical teams
2. **Tool Integration**: Tích hợp tools và external APIs
3. **Performance Optimization**: Tối ưu hóa hiệu suất và resource usage
4. **Production Deployment**: Triển khai hệ thống thực tế
5. **Monitoring & Observability**: Theo dõi và debug hệ thống multi-agent

### Thực hành thêm
- Thử tạo hệ thống với nhiều tác nhân hơn (4-5 agents)
- Triển khai parallel processing với multiple agents
- Tích hợp với external tools và APIs
- Xây dựng dashboard để monitor agent activities
- Thực hiện A/B testing với different agent configurations

**Chúc mừng! 🎉 Bạn đã hoàn thành notebook về Multi-Agent Systems với LangGraph!**